<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Convert-Label-Studio-CSV-to-Huggingface-autoTrain-Token-Classification-Training-File" data-toc-modified-id="Convert-Label-Studio-CSV-to-Huggingface-autoTrain-Token-Classification-Training-File-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Convert Label-Studio CSV to Huggingface autoTrain Token Classification Training File</a></span></li><li><span><a href="#Define-Predict-Function-Using-Trained-Model-From-AutoTrain-Huggingface-Returns-Dictionary-With-Data" data-toc-modified-id="Define-Predict-Function-Using-Trained-Model-From-AutoTrain-Huggingface-Returns-Dictionary-With-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define Predict Function Using Trained Model From AutoTrain Huggingface Returns Dictionary With Data</a></span></li><li><span><a href="#Make-list_of_predictions-using-HuggingFace-autoTrain-model" data-toc-modified-id="Make-list_of_predictions-using-HuggingFace-autoTrain-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Make list_of_predictions using HuggingFace autoTrain model</a></span></li><li><span><a href="#Interactively-Curate-True-and-Falsified-Predictions-and-store-to-joblib-file" data-toc-modified-id="Interactively-Curate-True-and-Falsified-Predictions-and-store-to-joblib-file-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Interactively Curate True and Falsified Predictions and store to joblib file</a></span></li><li><span><a href="#Backtest-agaist-real-data" data-toc-modified-id="Backtest-agaist-real-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Backtest agaist real data</a></span></li><li><span><a href="#Listen-to-Telegram-Channel-for-messages-and-predict-tokens" data-toc-modified-id="Listen-to-Telegram-Channel-for-messages-and-predict-tokens-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Listen to Telegram Channel for messages and predict tokens</a></span></li><li><span><a href="#Supplimentry-Functions" data-toc-modified-id="Supplimentry-Functions-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Supplimentry Functions</a></span></li></ul></div>

TODO:  
1 : make pytohn api for web interaction    
2 : test against other groups data than ratnakar  
3 : 

# Convert Label-Studio CSV to Huggingface autoTrain Token Classification Training File

In [ ]:
import pandas as pd
import ast
from tqdm import  tqdm
import math
import re
import numpy as np
import ipywidgets as widgets
from functools import partial
from IPython.display import clear_output
import joblib
import telegramAuth

##### Load Local saved Label-studio CSV export file

In [9]:
# pip install joblib

In [10]:
csv_import = pd.read_csv('./project-1-at-2022-09-19-18-31-ecd4d830.csv')
csv_import.head()

,text,id,label,annotator,annotation_id,created_at,updated_at,lead_time
0,MAHSEAMLES GIVEN AT 587 FOR DELIVERY TOUCHED ...,1127,"[{""start"": 0, ""end"": 10, ""text"": ""MAHSEAMLES"",...",1,1132,2022-09-15T17:30:47.590855Z,2022-09-15T17:30:47.590917Z,19.475
1,NIFTY FUTURES SUGGESTED AT 16390 ... TOUCHED 1...,1100,"[{""start"": 0, ""end"": 13, ""text"": ""NIFTY FUTURE...",1,1106,2022-09-14T17:19:43.922674Z,2022-09-15T17:30:12.437214Z,10.969
2,WATCHOUT.. FRIENDS... OUR NIFTY FUTURES GIVEN...,1098,"[{""start"": 27, ""end"": 40, ""text"": ""NIFTY FUTUR...",1,1104,2022-09-14T17:19:24.141919Z,2022-09-15T17:30:07.481390Z,16.964
3,INTRADAY UPDATE SELAN GIVEN ABOVE 202 TOUCHED...,1090,"[{""start"": 17, ""end"": 22, ""text"": ""SELAN"", ""la...",1,1096,2022-09-14T17:18:00.554760Z,2022-09-15T17:30:01.863693Z,11.138
4,WATCHOUT.. RELIANCE ABOVE 2776 TOUCHED 2785.80,1087,"[{""start"": 11, ""end"": 19, ""text"": ""RELIANCE"", ...",1,1093,2022-09-14T17:17:20.830008Z,2022-09-15T17:29:56.775820Z,9.455


In [11]:
# convert_label_studio_to_hngFace_autoTrain_dataset(csv_import)

In [12]:
def NEW_LOGIC_convert_label_studio_to_hngFace_autoTrain_dataset(csv_import) -> list:
    '''Converts export file from Label-Studio to HuggingFace autoTrain trainable dataset
    Example : 
    *****   INPUT of this function  **** (file : ../project-1-at-2022-09-13-06-14-0f0c0db3.csv))
    text	id	annotator	annotation_id	created_at	updated_at	lead_time	label
    KEEP   JAGSNPHARM ON RADAR... 	1001	1	1007	2022-09-13T05:51:47.578986Z	2022-09-13T05:51:47.579052Z	4.298	[{"start": 7, "end": 17, "text": "JAGSNPHARM", "labels": ["symbol"]}]
    INTRADAY : BUY JAGSNPHARM ABOVE 306 SL 302 TARGET 312 - 316 - 320 	1000	1	1006	2022-09-13T05:51:41.795524Z	2022-09-13T05:51:41.795587Z	6.055	[{"start": 15, "end": 25, "text": "JAGSNPHARM", "labels": ["symbol"]}, {"start": 32, "end": 35, "text": "306", "labels": ["enter"]}, {"start": 39, "end": 42, "text": "302", "labels": ["sl"]}, {"start": 50, "end": 53, "text": "312", "labels": ["exit"]}]
    SUPER DUPER FUNDAMENTALS IN JAGSNPHARM... 	999	1	1005	2022-09-13T05:51:34.283369Z	2022-09-13T05:51:34.283431Z	2.903	[{"start": 28, "end": 38, "text": "JAGSNPHARM", "labels": ["symbol"]}]
    *****   OUTPUT of this function  **** (file : ./dataset_for_huggingface_autoTrain_label_vlassification.csv)
    text	label
    ['KEEP   ', 'JAGSNPHARM', ' ON RADAR... ']	['NANA', 'symbol', 'NANA']
    ['INTRADAY : BUY ', 'JAGSNPHARM', ' ABOVE ', '306', ' SL ', '302', ' TARGET ', '312', ' - 316 - 320 ']	['NANA', 'symbol', 'NANA', 'enter', 'NANA', 'sl', 'NANA', 'exit', 'NANA']
    ['JAGSNPHARM', '... ']	['symbol', 'NANA']
    '''
    
    final_dataset = []
    for indx in csv_import.index:
        this_label = csv_import.label[indx]
        this_test_str = ''

    #     print(f'{indx=}')
    #     print(f'{this_label=}')

    #     print(f'{type(this_test_str)=}')
    #     print(f'{type(this_label)=}')

        if(type(this_test_str) != type(this_label)):
            if(np.isnan(this_label)):
                final_dataset.append({'text':'','label':'NANA'})
                continue

        this_literal_list = ast.literal_eval(this_label)

        start_pos_of_label__list = []
        for ipdata in this_literal_list:
            start_pos_of_label__list.append(ipdata['start'])
        start_pos_of_label__list
        start_pos_of_label__array = np.array(start_pos_of_label__list)
        sort_index = np.argsort(start_pos_of_label__array)
    #     print(start_pos_of_label__array)
    #     print(sort_index)

        this_text = csv_import.text[indx]

        final_label_text = []
        final_label_annotation = []
        init_labeling = True
        last_end_position = 0
        for i in sort_index:
        #     print(this_literal_list[i])
            this_label_type = this_literal_list[i]['labels'][0]
        #     print(f'{this_label_type=}')
            this_label_start_position = this_literal_list[i]['start']
            this_label_end_position = this_literal_list[i]['end']
        #     print(f'{this_label_start_position=}')
        #     print(f'{this_label_end_position=}')
        #     print(f'{this_text=}')
        #     print(f'{this_text[this_label_start_position:this_label_end_position]=}')
        #     print()

            if(init_labeling & this_label_start_position!=0):
                final_label_text.append(this_text[0:this_label_start_position])
                final_label_annotation.append('NANA')

            final_label_text.append(this_text[this_label_start_position:this_label_end_position])
            final_label_annotation.append(this_label_type)

            if((i)<sort_index[-1]):
                next_word_start_position = this_literal_list[i+1]['start']
                if(this_label_end_position!=next_word_start_position):
                    final_label_text.append(this_text[this_label_end_position:next_word_start_position])
                    final_label_annotation.append('NANA')


            last_end_position=this_label_end_position
        #     print('==debug==')
        #     print(f'{i=}')
        #     print(f'{sort_index[-1]=}')
        #     print(f'{this_label_end_position=}')
        #     print(f'{len(this_text)=}')
        #     print(f'{((i==sort_index[-1]) and this_label_end_position<len(this_text))=}')
        #     print('==debug==')

            if((i==sort_index[-1]) and this_label_end_position<len(this_text)):
                final_label_text.append(this_text[this_label_end_position:])
                final_label_annotation.append('NANA')


            init_labeling = False
        #     break

        # print('--------------------')
    #     display(f'{final_label_text=}')
    #     display(f'{final_label_annotation=}')

        final_dataset.append({'text':final_label_text,'label':final_label_annotation})
    return final_dataset

In [13]:
# this_dataset_df = pd.DataFrame(NEW_LOGIC_convert_label_studio_to_hngFace_autoTrain_dataset(csv_import))
# this_dataset_df.to_csv('./dataset_for_huggingface_autoTrain_label_vlassification_NEW_LOGIC.csv',index=False)
# this_dataset_df

In [14]:
# 192.168.227.62 - - [19/Sep/2022 18:09:36] "GET /api/jobs/65320ac73a4847a2b5a7f8ddbd45b821 HTTP/1.1" 200 -
# 2022-09-19 18:09:37 sparsify.blueprints.jobs INFO     getting job 65320ac73a4847a2b5a7f8ddbd45b821
# 2022-09-19 18:09:37 sparsify.blueprints.jobs INFO     retrieved job {'job': {'created': '2022-09-19T18:09:22.636614', 'worker_args': {'model_id': '5f276c2809654a24adb8d4ed4c3da3d7', 'profile_id': '4cbcefb9e2fe4a3fa7903604f72ea5a3', 'batch_size': 10, 'core_count': 8, 'pruning_estimations': True, 'quantized_estimations': False, 'iterations_per_check': 10, 'warmup_iterations_per_check': 5}, 'modified': '2022-09-19T18:09:36.205104', 'type_': 'CreatePerfProfileJobWorker', 'error': None, 'project_id': 'e1852305571c4f138149d12312acc30d', 'job_id': '65320ac73a4847a2b5a7f8ddbd45b821', 'progress': None, 'status': 'started'}}
# 192.168.227.62 - - [19/Sep/2022 18:09:37] "GET /api/jobs/65320ac73a4847a2b5a7f8ddbd45b821 HTTP/1.1" 200 -
# 2022-09-19 18:09:37 sparsify.workers.manager WARNING  Job 65320ac73a4847a2b5a7f8ddbd45b821 errored out Invalid input shape, cannot create a random input shape from: (None,)
# 2022-09-19 18:09:37 sparsify.blueprints.jobs INFO     getting job 65320ac73a4847a2b5a7f8ddbd45b821
# 2022-09-19 18:09:37 sparsify.blueprints.jobs INFO     retrieved job {'job': {'created': '2022-09-19T18:09:22.636614', 'worker_args': {'model_id': '5f276c2809654a24adb8d4ed4c3da3d7', 'profile_id': '4cbcefb9e2fe4a3fa7903604f72ea5a3', 'batch_size': 10, 'core_count': 8, 'pruning_estimations': True, 'quantized_estimations': False, 'iterations_per_check': 10, 'warmup_iterations_per_check': 5}, 'modified': '2022-09-19T18:09:37.427305', 'type_': 'CreatePerfProfileJobWorker', 'error': 'Invalid input shape, cannot create a random input shape from: (None,)', 'project_id': 'e1852305571c4f138149d12312acc30d', 'job_id': '65320ac73a4847a2b5a7f8ddbd45b821', 'progress': None, 'status': 'canceled'}}
# 192.168.227.62 - - [19/Sep/2022 18:09:37] "GET /api/jobs/65320ac73a4847a2b5a7f8ddbd45b821 HTTP/1.1" 200 -


# Define Predict Function Using Trained Model From AutoTrain Huggingface Returns Dictionary With Data

In [15]:
# pip install PyTorch

In [16]:
###### LOAD PRETRAINED MODEL FROM HUGGINGFACE autoTrain #################
from transformers import AutoModelForTokenClassification, AutoTokenizer
model = AutoModelForTokenClassification.from_pretrained("hemangjoshi37a/autotrain-ratnakar_1000_sample_curated-1474454086", use_auth_token=True)
tokenizer = AutoTokenizer.from_pretrained("hemangjoshi37a/autotrain-ratnakar_1000_sample_curated-1474454086", use_auth_token=True)

In [17]:
# #### EXPORT HUGGINFFACE MODEL TO ONNX TYPE MODEL #####
# base_model=model

# from transformers.models.distilbert import DistilBertConfig, DistilBertOnnxConfig

# DistilBertOnnxConfig(base_model.config)

# from pathlib import Path
# from transformers.onnx import export
# from transformers import AutoTokenizer, AutoModel

# onnx_path = Path("./autotrain-ratnakar_1000_sample_curated-1474454086")
# # model_ckpt = "distilbert-base-uncased"
# # base_model = AutoModel.from_pretrained(model_ckpt)
# # tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# onnx_inputs, onnx_outputs = export(tokenizer, base_model, DistilBertOnnxConfig(base_model.config), DistilBertOnnxConfig(base_model.config).default_onnx_opset, onnx_path)

# from transformers import AutoConfig

# config = AutoConfig.from_pretrained('hemangjoshi37a/autotrain-ratnakar_1000_sample_curated-1474454086')

# tokenizer.save_pretrained('./model_files/')
# config.save_pretrained('./model_files/')

In [18]:
class_number_to_name_dict = {0 : '',
                            1 : 'btst',
                            2 : 'delivery',
                            3 : 'enter',
                            4 : 'momentum',
                            5 : 'exit',
                            6 : 'exit2',
                            7 : 'exit3',
                            8 : 'intraday',
                            9 : 'sl',
                            10 : 'symbol',
                            11 : 'touched'}
# class_number_to_name_dict

In [19]:
def get_class_map_from_message_NEW(input_message:str) -> dict:
    ########### PREDICT TEXT AND CLASSIFY WORDS ##########
    ip1 = tokenizer(input_message,return_tensors='pt')
    op1 = model(**ip1)

    current_word = ''
    sentence = []
    sentence_class= []
    sentence_class_name= []
    list_of_decoded_words = tokenizer.batch_decode(ip1['input_ids'][0])
    last_word_contained_hash = False
    last_classification_numner = 0
    last_decoded_word = ''

    for onet in range(len(ip1['input_ids'][0])):
        this_token = ip1['input_ids'][0][onet]
        this_classification = op1.logits[0][onet].tolist()
        this_decoded_word = list_of_decoded_words[onet]
        this_classification_number = np.argmax(this_classification)

        if(this_decoded_word=='[CLS]' or this_decoded_word=='[SEP]'):
            continue

#         print(f'{this_decoded_word=}')
# #         print(f'{this_classification=}')
#         print(f'{this_classification_number=}')

        this_word_contains_hash= '#' in this_decoded_word

        if('#' in this_decoded_word):
            hash_replaced_word = this_decoded_word.replace('#','')
#             print(f'''{hash_replaced_word=}''')
            current_word = current_word+hash_replaced_word
#             print(f'{current_word=}')
            last_word_contained_hash=True

        elif((this_classification_number==last_classification_numner) and ((this_decoded_word=='.') or (last_decoded_word=='.'))):
            last_classification_numner = this_classification_number
            current_word = current_word+this_decoded_word
                
        else:
#             print('========== insidious ===============')
            sentence.append(current_word)
            sentence_class.append(last_classification_numner)
            sentence_class_name.append(class_number_to_name_dict[last_classification_numner])
#             print(f'{current_word=}')
#             print(f'{sentence=}')
#             print(f'{last_classification_numner=}')
#             print(f'{sentence_class=}')
#             print(f'{current_word=}')
            current_word=this_decoded_word
            last_classification_numner = this_classification_number
            
            last_word_contained_hash=False
        last_decoded_word = this_decoded_word


#         print('======================================')

    sentence.append(current_word)
    sentence_class.append(last_classification_numner)
    sentence_class_name.append(class_number_to_name_dict[last_classification_numner])
    resultsdf = pd.DataFrame({'sentence':sentence,
                        'sentence_class':sentence_class,
                        'sentence_class_name':sentence_class_name,
                       })
    
#     display(resultsdf)
    return resultsdf

In [20]:
get_class_map_from_message_NEW('''[ Photo ]
FUNDAMENTALS''')

,sentence,sentence_class,sentence_class_name
0,,0,
1,[,0,
2,Photo,0,
3,],0,
4,FUNDAMENTALS,0,


# Make list_of_predictions using HuggingFace autoTrain model

In [23]:
list_of_predictions = joblib.load('./list_of_predictions.p')
# list_len = len(list_of_predictions)
# # len_of_messagedf = len(messagedf.index)
# print(f'{list_len=}')
# # print(f'{len_of_messagedf=}')

In [24]:
# list_of_predictions = []
# for one_message in tqdm(messagesdf.message):
# #     print(one_message)
#     this_pred = get_class_map_from_message_NEW(one_message)
#     list_of_predictions.append(this_pred)
# #     print(list_of_predictions)
# #     break

In [25]:
# joblib.dump(list_of_predictions,'./list_of_predictions.p')

# Interactively Curate True and Falsified Predictions and store to joblib file

In [26]:
##### Load messagedf using joblib #######
messagesdf = joblib.load('./curated_predictions.p')
# joblib.dump(messagesdf,'./curated_predictions.p')
sample_number = joblib.load('./sample_number.p')['sample_number'][0]
# joblib.dump(pd.DataFrame({'sample_number':sample_number},index=[0]),'./sample_number.p')
messagesdf

,time,message,symbol,predictions,isPredAccurate
0,2022-09-08 06:50:40,FRIENDS..\n\nANYTIME MOMENTUM CAN COME IN DCMN...,DCMNVL,0 1 FRIENDS..ANYTIME ...,True
1,2022-09-08 06:42:16,KEEP DCMNVL ON RADAR\n\nAND WAIT TILL THERE IS...,DCMNVL,0 1 KEEP 2 DCMNVL 3 ...,True
2,2022-09-08 06:35:28,DCMNVL... DCM Nouvelle Ltd\n\nHUGE MOMENTUM AB...,DCMNVL,0 1 DCMNVL 2 ...,True
3,2022-09-08 06:33:50,SUPER DUPER FUNDAMENTALS,SUPER,0 1 SUPER 2 ...,True
4,2022-09-08 06:32:47,DELIVERY\n\nBUY DCMNVL CMP 202.70 SL 175 TARGE...,DCMNVL,0 1 DELIVERY 2 BUY 3...,False
...,...,...,...,...,...
11719,2019-04-04 04:17:21,INTRADAY : BUY GUJGASLTD ABOVE 153 SL 150 TAR...,GUJGASLTD,0 1 INTRADAY 2 ...,NaN
11720,2019-04-04 04:12:08,DELIVERY : BUY BOMDYEING CMP 144.85 SL 134 TAR...,BOMDYEING,0 1 DELIVERY 2 ...,NaN
11721,2019-04-03 09:58:55,BTST : MMTC CMP 29.25,MMTC,0 1 BTST 2 : 3 MMTC 4 ...,NaN
11722,2019-04-03 09:55:59,BTST : EMKAY CMP 110,EMKAY,0 1 BTST 2 : 3 EMKAY 4 ...,NaN


In [27]:
#### Sample Number Selector Carousal Widget ###
def carousel_box_color(i,sample_number) -> str:
    if(i==sample_number):
        return 'danger'
    else:
        return 'warning'
def carousel_on_sample_number_click(widget_data):
    global sample_number
    clear_output(wait=True)
#     print(widget_data)
#     print(int(widget_data.description))
    sample_number = int(widget_data.description)
#     print(f'{sample_number=}')
    display_annotations()
    display_common_widget()
    sample_number+=1
    
def carousal_number_range(sample_number) -> range:
    if(sample_number>40):
        return range(sample_number-20,sample_number+20)
    else:
        return range(50)

def display_sample_number_selector_carousal(sample_number):
    item_layout = widgets.Layout(height='25px', min_width='60px')
    items = [widgets.Button(layout=item_layout, description=str(i), button_style=carousel_box_color(i,sample_number)) for i in carousal_number_range(sample_number)]
    box_layout = widgets.Layout(overflow='scroll hidden',
                        border='3px solid black',
                        width='700px',
                        height='',
                        flex_flow='row',
                        display='flex')
    sample_number_selector_carousel = widgets.Box(children=items, layout=box_layout,)
    for each_item in  items:
        each_item.on_click(partial(carousel_on_sample_number_click))
    display(sample_number_selector_carousel)

# display_sample_number_selector_carousal(sample_number)

In [30]:
####### Fx for displaying widget and predicted annotations #########
def display_annotations():
    global messagesdf
    print(messagesdf.message[sample_number])
    
    this_predictions=list_of_predictions[sample_number]
    # this_predictions = get_class_map_from_message_NEW(messagesdf.message[sample_number])
    ## inserting value in cell [index(messagedf_index),prediction(coumn_name)]
    messagesdf.iloc[sample_number,3]=this_predictions
    joblib.dump(messagesdf,'./curated_predictions.p')
    joblib.dump(pd.DataFrame({'sample_number':sample_number},index=[0]),'./sample_number.p')
    display(this_predictions)

def display_common_widget():
    print(f'----- {sample_number} -----')
    display(widgets.HBox([previous_button,false_pred_button,true_pred_button, next_button]))
    if(sample_number<=5):
        display(messagesdf.head(9))
    if(sample_number>5):
        display(messagesdf[sample_number-4:sample_number+5])
    display_sample_number_selector_carousal(sample_number)

####### Define onClick functions for buttons #########
def true_prediction_onclick(ipip):
    global sample_number,messagesdf
    clear_output(wait=True)
    messagesdf.iloc[sample_number,4]=True
    sample_number+=1
    ## inserting value in cell [index(messagedf_index),isPredAccurate(coumn_name)]
    display_annotations()
    display_common_widget()
    
def false_prediction_onclick(ipip):
    global sample_number,messagesdf
    clear_output(wait=True)
    messagesdf.iloc[sample_number,4]=False
    sample_number+=1
    ## inserting value in cell [index(messagedf_index),isPredAccurate(coumn_name)]
    display_annotations()
    display_common_widget()
    
def previous_button_onclick(ipip):
    global sample_number
    sample_number-=1
    clear_output(wait=True)
    display_annotations()
    display_common_widget()

def next_button_onclick(ipip):
    global sample_number
    sample_number+=1
    clear_output(wait=True)
    display_annotations()
    display_common_widget()

In [31]:
####### Define button instances ########
true_pred_button = widgets.Button(
    description='True Prediction',
    disabled=False,
    button_style='success',
    tooltip='Click me',
    icon='check'
)
true_pred_button.on_click(partial(true_prediction_onclick))

false_pred_button = widgets.Button(
    description='False Prediction',
    disabled=False,
    button_style='danger',
    tooltip='Click me',
    icon='close'
)
false_pred_button.on_click(partial(false_prediction_onclick))

previous_button = widgets.Button(
    description='⏮️ Previous',
    disabled=False,
    button_style='info', 
    tooltip='Click me',
)
previous_button.on_click(partial(previous_button_onclick))


next_button = widgets.Button(
    description='Next ⏭️',
    disabled=False,
    button_style='info',
    tooltip='Click me',
)
next_button.on_click(partial(next_button_onclick))

display_annotations()
display_common_widget()

REFEX IS TRYING TO BREAKOUT

OPEN - LOW IS SAME.. WHICH IS A BULLISH SIGN 


,sentence,sentence_class,sentence_class_name
0,,0,
1,REFEX,10,symbol
2,IS,0,
3,TRYING,0,
4,TO,0,
5,BREAKOUT,0,
6,OPEN,0,
7,-,0,
8,LOW,0,
9,IS,0,


----- 297 -----


,time,message,symbol,predictions,isPredAccurate
293,2022-08-26 06:46:30,DON'T MISS,NaN,"[, DON, ', T, MISS]",True
294,2022-08-26 06:46:25,AUSOMENT...,AUSOMENT,"[, AUSOMENT, ...]",True
295,2022-08-26 06:43:34,FRIENDS.. EYES ON AUSOMENT \n\nABOVE 82 IS A F...,AUSOMENT,"[, FRIENDS..EYES, ON, AUSOMENT, ABOVE, 82, IS,...",True
296,2022-08-26 06:39:54,WATCHOUT FRIENDS...\n\nREFEX GIVEN FOR BUY AT ...,REFEX,"[, WATCHOUT, FRIENDS..., REFEX, GIVEN, FOR, BU...",False
297,2022-08-26 06:22:48,REFEX IS TRYING TO BREAKOUT\n\nOPEN - LOW IS S...,REFEX,"[, REFEX, IS, TRYING, TO, BREAKOUT, OPEN, -, L...",NaN
298,2022-08-26 06:16:16,FLAG BREAKOUT\n\nINTRADAY :BUY REFEX ABOVE 13...,REFEX,0 1 FLAG 2 BREAKOUT 3...,NaN
299,2022-08-26 05:58:44,ZUARI : SAME PATTERN AS KUANTUM AND SEE IT DID...,ZUARI,0 1 ZUARI 2 ...,NaN
300,2022-08-26 05:50:32,IF IT SUSTAINS ABOVE 93 THEN CAN MOVE UPTO 104...,NaN,0 1 IF 2 IT 3...,NaN
301,2022-08-26 05:46:42,PATTERN OF INVERTED H&S\n\nA CLOSE ABOVE 93 LE...,NaN,0 1 PATTERN 2 ...,NaN


Box(children=(Button(button_style='warning', description='277', layout=Layout(height='25px', min_width='60px')…

In [37]:
# pip install pyotp

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


# Backtest agaist real data

In [38]:
########### IMPORT LIBRARIES ##############
import datetime
# pd.set_option('plotting.backend', 'pandas_bokeh')
# import pandas_bokeh
# pandas_bokeh.output_notebook()
from kiteconnect import  exceptions
import login_new
kite = joblib.load('./kitefile.p')
try:
    display(kite.profile()['user_name'])
except Exception as tknxcp:
    print(tknxcp)
    kite = login_new.login_haj_new()
    display(kite.profile()['user_name'])

Incorrect `api_key` or `access_token`.


'Hemang Atulkumar Joshi'

'Hemang Atulkumar Joshi'

In [ ]:
droppedna = messagesdf.dropna()
true_predictions_df = droppedna.loc[droppedna.isPredAccurate]
#### Time correct Time column
true_predictions_df.time =  true_predictions_df.time + datetime.timedelta(minutes=29, hours=5)
true_predictions_df

In [ ]:
#### extracting onyl those rows data which has all 4 values in predictions
#### 1:symbol, 2:enter, 3:sl, 4:exit
timeOfPredToClassDataList = [] # list to create time based dataframe of classification
for index,data in tqdm(true_predictions_df.iterrows()):
#     print(index)
    this_prediction = get_class_map_from_message_NEW(data.message)
#     display(this_prediction)
    check_if_all_val_available_in_prediction = set([2,3,5,9,10]).issubset(set(this_prediction.sentence_class.tolist())) or   set([8,3,5,9,10]).issubset(set(this_prediction.sentence_class.tolist())) 
#     print(check_if_all_val_available_in_prediction)
    if(check_if_all_val_available_in_prediction):
        this_symbol = this_prediction.loc[this_prediction['sentence_class_name'] == 'symbol']['sentence'].tolist()[0]
        this_enter = float(this_prediction.loc[this_prediction['sentence_class_name'] == 'enter']['sentence'].tolist()[0])
        this_sl = float(this_prediction.loc[this_prediction['sentence_class_name'] == 'sl']['sentence'].tolist()[0])
        this_exit = float(this_prediction.loc[this_prediction['sentence_class_name'] == 'exit']['sentence'].tolist()[0])
        print(f'''Entry time = {data['time']}''')
        print(f'{this_symbol=}')
        print(f'{this_enter=}')
        print(f'{this_sl=}')
        print(f'{this_exit=}')
        try:
            this_insttkn = kite.ltp(f'NSE:{this_symbol}')[f'NSE:{this_symbol}']['instrument_token']
            timeOfPredToClassDataList.append({'time':data['time'],
                                              'symbol':this_symbol,
                                              'insttkn':this_insttkn,
                                              'enter': this_enter,
                                              'sl':this_sl,
                                              'exit':this_exit
                                             })
        except KeyError as ke:
            print(f'key error for : {this_symbol} : {ke}')

In [ ]:
extracted_data_df = pd.DataFrame(timeOfPredToClassDataList)
extracted_data_df

In [ ]:
# # extracted_data_df = extracted_data_df.loc[::-1, :] # reverse row order
# extracted_data_df

In [ ]:
extracted_data_df['exit_time_in_backtest'] = datetime.datetime.now()
extracted_data_df['exit_type_in_backtest'] = 'not_yet'
extracted_data_df['exit_val_in_backtest'] = 0.0
extracted_data_df['enter_val_in_backtest'] = 0.0
extracted_data_df['enter_time_in_backtest'] = 0.0
extracted_data_df['gained_percent'] = 0.0
extracted_data_df

In [ ]:
##### Determine exit types (SL or TGT) if exited and plot gained_percent graph ####
pd.options.mode.chained_assignment = None  # default='warn'

nest = []
nest_symbol_list = []
for indx,data in  tqdm(extracted_data_df.iterrows()):
    ### getting historical data ###
    hist_raw_data = kite.historical_data(data['insttkn'],data['time'],data['time']+datetime.timedelta(days=60),'minute')
    histdf = pd.DataFrame(hist_raw_data)
    histdf.date=histdf.date.dt.tz_localize(None)
#     print(data['symbol'])

    histdf_from_message_date = histdf.loc[pd.to_datetime(histdf['date']).dt.date==data['time'].date()]
    enterdf = histdf_from_message_date.loc[histdf_from_message_date['high']>data['enter']]

    if(enterdf.empty!=True):
        enter_price = enterdf.close.tolist()[0]
        enter_time = enterdf.date.tolist()[0]
#         print(f'{enter_price=}')
#         print(f'{enter_time=}')
        extracted_data_df['enter_val_in_backtest'][indx] = enter_price
        extracted_data_df['enter_time_in_backtest'][indx] = enter_time

        ##### determining when TGT and SL hit ####
        final_exit_time = datetime.datetime.now()
        tgt_exit_time = datetime.datetime.now()
        sl_exit_time = datetime.datetime.now()
        final_exit_type = 'not_exited_yet'
        tgt_exit_val = 0.0
        sl_exit_val = 0.0
        final_exit_val = 0.0

        try:
            tgt_exit_df = histdf.loc[histdf.close>data['exit']]
            tgt_exit_time = tgt_exit_df.iloc[0]['date']
            tgt_exit_val = tgt_exit_df.close.iloc[-1]
    #         print(f'{tgt_exit_time=}')
    #         print(f'{tgt_exit_val=}')
        except IndexError as ie:
            pass

        try:
            sl_exit_df = histdf.loc[histdf.close<data['sl']]
            sl_exit_time = sl_exit_df.iloc[0]['date']
            sl_exit_val = sl_exit_df.close.iloc[-1]
    #         print(f'{sl_exit_time=}')
    #         print(f'{sl_exit_val=}')
        except IndexError as ie:
            pass

        ##### determining if exited with TGT or SL by comparing what occured earlier ###
        if(tgt_exit_time<sl_exit_time):
            final_exit_time = tgt_exit_time
            final_exit_type = 'tgt'
            final_exit_val = tgt_exit_val
        else:
            final_exit_time = sl_exit_time
            final_exit_type = 'sl'
            final_exit_val = sl_exit_val

        ### storing final values in extracted_data_df ###
        extracted_data_df['exit_time_in_backtest'][indx] = final_exit_time
        extracted_data_df['exit_type_in_backtest'][indx] = final_exit_type
        extracted_data_df['exit_val_in_backtest'][indx] = final_exit_val

    #     print(f'{final_exit_time=}')
        final_histdf = histdf.loc[histdf.date<final_exit_time]

        if(final_histdf.empty != True):
            #### determining percent gained ###
            final_histdf['percent_gain'] = 100*(final_histdf.close-enter_price)/enter_price

            nest.append(final_histdf.percent_gain.tolist())
            nest_symbol_list.append(data['symbol'])

            final_percent_gained = final_histdf['percent_gain'].tolist()[-1]
            extracted_data_df['gained_percent'][indx] = final_percent_gained
        #     histdf[['date','percent_gain']].plot_bokeh(x='date')
extracted_data_df

In [ ]:
import matplotlib as pyplot

In [ ]:
#### Compare gains to Nifty 50 ####
nifty_raw = kite.historical_data(256265,extracted_data_df.time.iloc[-1],extracted_data_df.time.iloc[-1]+datetime.timedelta(days=60),'minute')
nifty_df = pd.DataFrame(nifty_raw)
nifty_gained_percent_in_60days =  100*(nifty_df.close.iloc[-1] - nifty_df.close[0])/nifty_df.close[0]
print(f'{nifty_gained_percent_in_60days=}')

total_gained_percent_by_all_trades = extracted_data_df.gained_percent.sum()
print(f'{total_gained_percent_by_all_trades=}')
total_number_of_trades_done = 2*len(extracted_data_df.index)
print(f'{total_number_of_trades_done=}')

### Calculate compunded gains ####
init_percent = 100.0
list_map = []
for one_percent in extracted_data_df.gained_percent.tolist()[::-1]:
    init_percent = init_percent*((100+one_percent)/100)
    list_map.append({'init_percent':init_percent,'one_percent':one_percent})
#     print(one_percent)
compounded_gains = init_percent-100
print(f'{compounded_gains=}')
pd.DataFrame(list_map).plot()

In [ ]:
pd.DataFrame(list_map).plot()

In [ ]:
extracted_data_df['gained_percent'].plot.bar()

In [ ]:
final_percent_gain_df = pd.DataFrame(nest, nest_symbol_list).T
final_percent_gain_df

In [ ]:
### add counter number at last in repeating column names
new_column_name_list = []
counter = 0
for one_name in final_percent_gain_df.keys().tolist():
    if(one_name not in new_column_name_list):
        new_column_name_list.append(one_name)
    else:
        new_column_name_list.append(one_name+'_'+str(counter))
        counter+=1
        
final_percent_gain_df.columns = new_column_name_list
final_percent_gain_df

# Listen to Telegram Channel for messages and predict tokens

In [ ]:
# Code from  https://python.plainenglish.io/telegram-channel-listener-with-python-8176ebe3c89b
import configparser
import json
import re
from telethon.errors import SessionPasswordNeededError
from telethon import TelegramClient, events, sync
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (PeerChannel)

api_id = telegramAuth.api_id
api_hash = telegramAuth.api_hash
phone = telegramAuth.phone

user_input_channel = 'https://t.me/testchannelnottaken'

client = TelegramClient(phone, api_id, api_hash)

await client.connect()

if not await client.is_user_authorized():
    await client.send_code_request(phone)
    await client.sign_in(phone, input('Enter the code: '))
else :
    print('Autorised')

# Listen to messages from target channel 

@client.on(events.NewMessage()) 
async def newMessageListener(event):
    # Get message text 
    newMessage = event.message.message 
    classified_dict =  get_class_map_from_message_NEW(newMessage)
    print(classified_dict)
# #### GET FULL MESSAGE EVENT OBJECT in ASYNC MAnner #####
# @client.on(events.NewMessage)
# async def my_event_handler(event):
#     print('{}'.format(event))

In [ ]:
##### Start Listening to Messages ########
async with client: 
    print('Now wating for new messages...')
    await client.run_until_disconnected() 

# Supplimentry Functions

In [ ]:
### JOIN CHANNEL FROM TELEGRAM CLIENT (not from telepot BOT) ####
# from telethon.tl.functions.channels import JoinChannelRequest
# await client(JoinChannelRequest('https://t.me/INTRADAY_001'))

#### TELEGRAM BOT #####
# import telepot
# bot_tkn = telegramAuth.bot_tkn
# bt1 = telepot.Bot(bot_tkn)
# response = bt1.getUpdates()
# print(response)

# #### GET FULL MESSAGE EVENT OBJECT in ASYNC MAnner #####
# @client.on(events.NewMessage)
# async def my_event_handler(event):
#     print('{}'.format(event))
# await client.start()
# await client.run_until_disconnected()